In [ ]:
from IPython.core.display import Image
from IPython import display

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal, norm
import pandas as pd

In [ ]:
class stump(object):
    def __init__(self, v, u, x, y):
        self.v = v
        self.u = u
        cuales_izq = np.where(x[:,v] <= u)[0]
        if np.mean(y[cuales_izq]) < 0:
            self.y_izq = -1
        else:
            self.y_izq = 1
        cuales_dcha = np.where(x[:,v] > u)[0]
        if np.mean(y[cuales_dcha]) < 0:
            self.y_dch = -1
        else:
            self.y_dch = 1
    def predict(self, x):
        n = len(x)
        output = self.y_dch * np.ones(n)
        cuales_izq = np.where(x[:,self.v] <= self.u)[0]
        output[cuales_izq] = self.y_izq
        return output

In [ ]:
def load_spam():
  data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data',header=None)
  data.columns=["wf_make",         
"wf_address",      
"wf_all",          
"wf_3d",           
"wf_our",          
"wf_over",         
"wf_remove",       
"wf_internet",     
"wf_order",        
"wf_mail",         
"wf_receive",      
"wf_will",         
"wf_people",       
"wf_report",       
"wf_addresses",    
"wf_free",         
"wf_business",     
"wf_email",        
"wf_you",          
"wf_credit",       
"wf_your",         
"wf_font",         
"wf_000",          
"wf_money",        
"wf_hp",           
"wf_hpl",          
"wf_george",       
"wf_650",          
"wf_lab",          
"wf_labs",         
"wf_telnet",       
"wf_857",          
"wf_data",         
"wf_415",          
"wf_85",           
"wf_technology",   
"wf_1999",         
"wf_parts",        
"wf_pm",           
"wf_direct",       
"wf_cs",           
"wf_meeting",      
"wf_original",     
"wf_project",      
"wf_re",           
"wf_edu",          
"wf_table",        
"wf_conference",   
"cf_;",            
"cf_(",            
"cf_[",            
"cf_!",            
"cf_$",            
"cf_#",            
"cap_average", 
"cap_longest", 
"cap_total",
"target"]
  return data

# Introducción al aprendizaje Automático (2)

### Fundamentos de Aprendizaje Automático 

### Enero 2025

**Emilio Parrado Hernández, Vanessa Gómez Verdejo, Pablo Martínez Olmos**

Departamento de Teoría de la Señal y Comunicaciones

**Universidad Carlos III de Madrid**

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />

# Contenidos que se van a tratar a lo largo de la sesión

- Introducción a scikit learn
- Método de los k vecinos más próximos
- Árboles de decisión y regresión
- Introducción a métodos *ensemble* (*random forest*)
- Parámetros entrenables e hiperparámetros
- Funciones de coste y de pérdidas
- Sobreajuste
- Evaluación de modelos para clasificación y regresión
- Muy centrados en el uso de algoritmos, no en su optimización


<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />

# Método de los $k$ vecinos más próximos
- En la sesión anterior visitamos el método del vecino más próximo como paradigma de **modelo no paramétrico**
- Es una particularización de un método más general que se conoce como *kNN* ($k$ *nearest neighbors*)
- Considerar los $k$ vecinos más próximos permite definir el vecindario de una manera **más robusta**
- $k$ es un **hiperparámetro** que se fija mediante **conocimiento a priori**, por tanto $k$NN se sigue considerando **no paramétrico**
- El método de los $k$ vecinos puede emplearse tanto para **clasificación** como para **regresión**

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Clasificación con $k$NN

### 2. Clasificación de una observación de test $\mathbf x_t$
- Buscar de entre las observaciones del conjunto de entrenamiento las $k$ que están a una **distancia** menor de $\mathbf x_t$: Estas muestras son los $k$ vecinos más próximos de $\mathbf x_t$
- Rescatar los *targets*, en este caso la **clase de pertenencia** de cada uno de los $k$ vecinos. Fijaos que esta metodología sirve para clasificaciones **binarias y multiclase**
- Asignar como *target* para la observación de test, $y_t$, a la clase **mayoritaria** de entre los $k$ vecinos. 

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


   
### 1. Construcción del modelo
- No tenemos modelo, sólo una **tabla** que guarda las observaciones del **conjunto** de entrenamiento y sus correspondientes *targets*
- Definir una **distancia** o **métrica** para comparar observaciones, típicamente la distancia **euclídea** o **norma $L_2$**
$$
d(\mathbf x_1,\mathbf x_2) = \sqrt{\sum_{j=1}^d((\mathbf x_1)_j - (\mathbf x_2)_j)^2} = \sqrt{(\mathbf x_1 - \mathbf x_2)^\top(\mathbf x_1 - \mathbf x_2)}
$$  
en python `d_x1_x2 = numpy.linalg.norm(x1-x2)` 
- Fijar $k$ (número de vecinos)
- Fijar una estrategia de voto para determinar la clase mayoritaria entre los $k$ vecinos:
  - Cada vecino emite un voto unitario en favor de su clase
  - El voto de cada vecino se **pondera inversamente proporcional** a la distancia a la que se encuentre de $\mathbf x_t$, de este modo los vecinos más cercanos influyen más 

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


### 3. Ejemplo de funcionamiento con un *toy problem*

In [ ]:
from sklearn.datasets import make_classification

x,y = make_classification(n_samples=20, 
                          n_features=2,
                          n_informative=2, 
                          n_redundant=0,
                          n_repeated=0, 
                          n_classes=2, 
                          n_clusters_per_class=2,
                          weights=None, 
                          flip_y=0.01, 
                          class_sep=1.0, 
                          hypercube=False, 
                          shift=0.0, 
                          scale=1.0, 
                          shuffle=True, 
                          random_state=42)

In [ ]:
xt = np.array([-0.2, 0.7])

In [ ]:
def pinta_ejemplo(x,y,xt,ax):
  ax.scatter(x[y==0,0],x[y==0,1],color='blue',marker='o')
  ax.scatter(x[y==1,0],x[y==1,1],color='red',marker='x')
  ax.scatter(xt[0], xt[1], marker='s', color='green')
fx,ax = plt.subplots(1,1,figsize=(7,7))
pinta_ejemplo(x,y,xt,ax)

In [ ]:
from sklearn.metrics import pairwise_distances 
distancias = pairwise_distances(xt.reshape(1,-1), x,'euclidean') 

# Ordenar todos los vecinos para cada observación de test
id_vecinos = np.argsort(distancias) # indices de las observaciones de entrenamiento, ¿quién es el veino k-ésimo?

sorted_distancias = 1e-6+np.sort(distancias) # distancias a los vecinos ¿qué tan lejos está el vecino k-ésimo?

# clase a la que vota cada vecino
clase_vecinos = y[id_vecinos]
# peso con el que vota cada vecino a la clase de la observación de test
peso_vecinos = 1./sorted_distancias



In [ ]:
def pinta_radios(c,x,ax):
  for ii in range(len(x)):
    ax.plot([c[0],x[ii,0]],
            [c[1],x[ii,1]],
            linestyle=':',
            linewidth=2)
    
def pinta_votos(clase, peso, ax):
  labels = ['unitario', 'ponderado']
  k = len(clase)
  bottom = np.zeros(2)
  width = 0.5
  for ii in range(k):
    if clase[ii] == 1:
      color = 'red'
    else:
      color = 'blue'
    ax.bar(labels, [1./k, peso[ii]], 
           width=width, 
           label='vecino {0:d}'.format(ii+1),
           bottom = bottom,
           edgecolor='black',
           color=color)
    bottom += np.array([1./k, peso[ii]])

In [ ]:
v_k = np.array([1,2,4,8])
ii=7
fx,ax = plt.subplots(len(v_k),2,figsize=(2*ii,len(v_k)*ii))
for ik,kk in enumerate(v_k):
  ax[ik][0].set_title('k = {0:d} vecinos'.format(kk))
  ax[ik][1].set_title('votos de los {0:d} vecinos'.format(kk))
  pinta_ejemplo(x,y,xt,ax[ik][0]) 
  pinta_radios(xt,np.array([x[cc,:] for cc in id_vecinos[0,:kk]]),ax[ik][0])
  pinta_votos(clase_vecinos[0,:kk], peso_vecinos[0,:kk]/np.sum(peso_vecinos[0,:kk]), ax[ik][1])

# Problema de clasificación para esta sesión: spam detection

## Repositorio de bases de datos de UCI
Es uno de los problemas clásicos del [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php). En general la manera de comparar las prestaciones de algoritmos de aprendizaje es evaluar su capacidad para construir modelos en problemas *benchmark*. Existe otra aproximación, el campo de *Statistical Learning Theory* (SLT), que trata de analizar estos comportamientos mediante cotas en la capacidad de generalización de estos algoritmos, al estilo de las cotas de Shannon en Teoría de la Información. A efectos prácticos, si bien la aproximación SLT parece más robusta, la experiencia real es que las estimaciones de prestaciones basadas en *benchmarks* se parecen más a lo que experimentan los usuarios de estos algoritmos al ponerlos en práctica con datos reales.

El repositorio de bases de datos de UCI es uno de los más empleados en el diseño de algoritmos de propósito general y en los programas de formación de científicos de datos, ya que abarca problemas supervisados y no supervisados de muy diversa índole y permite adquirir intuiciones acerca del funcionamiento de estos algoritmos en situaciones diversas.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Spam detection database
- 4601 observaciones (1813 spam, 39.4%)
- target binario: spam ($y=1$) o no spam ($y=0$)
- No disponemos de los textos, las observaciones están formadas por 57 variables continuas
  - 48 variables reales continuas en el intervalo [0,100] *word_freq_WORD*: frecuencia de aparición de la palabra *WORD* en el correo (en porcentaje)
  - 6 variables reales continuas en el intervalo [0,100] *char_freq_CHAR*: frecuencia de aparición del caracter *CHAR* en el correo (porcentaje)
  - 1 variable real continua: longitud promedio de secuencias de caracteres en mayúscula ininterrumpidos
  - 1 variable entera continua: longitud de la secuencia de mayúsculas ininterrumpida más larga
  - 1 variable entera continua: número total de mayúsculas en el correo
  
<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


### Consideraciones sobre la base de datos
- La base es de junio-julio de 1999
  - El NLP no estaba tan desarrollado como ahora, 
    - Los correos electrónicos eran básicamente texto ASCII
    - así que buscan estadísticos sobre repeticiones de palabras y caracteres de puntuación.
- El dueño de la base de datos se llama George
- Trabajan en Hewlett Packard
  - Pueden buscar códigos de área y palabras clave dentro de la organización y de los contactos y/o proyectos de George
  
<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Proceso de carga de datos
Pasos que tenemos que dar hasta poder emplear el algoritmo de entrenamiento
1. Leer los datos
2. Separar observaciones de *targets*
3. Dividir entre conjunto de entrenamiento y conjunto de test

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


### 1. Lectura de los datos
- Importar los datos desde disco. 
- En este caso es un csv, con lo que lo cargaremos en **pandas** para poder ver alguna descripción de las observaciones

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
data = load_spam()
print("Cargadas {0:d} observaciones con {1:d} columnas\n".format(len(data), len(data.columns)))
print("Ejemplos")
data.tail()

In [ ]:
data.describe()

### 2. Separar observaciones de *targets*
Leyendo la documentación de la base de datos vemos que el *target* es la última columna de cada registro de la tabla.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
X = data[data.columns[:57]].values
Y = data['target'].values
print("{0:d} observaciones con {1:d} columnas".format(X.shape[0], X.shape[1]))
print("{0:d} targets".format(len(Y)))
print("Valores de los targets:")
print(np.unique(Y))

### 3. Dividir entre conjunto de entrenamiento y conjunto de test
El éxito de un modelo de aprendizaje automático se mide en su capacidad de generalización procesando datos que no fueron empleados durante el entrenamiento. Para ello necesitamos disponer de dos conjuntos disjuntos de datos:
- el **conjunto de entrenamiento**, que son las observaciones que se le pasan al **algoritmo de entrenamiento** (al método `fit`) para que éste optimice el modelo
- el **conjunto de test**, es un conjunto separado que se procesa con el **modelo  ya entrenado** y que usamos para medir la **capacidad de generalización** (con el método `score`)). Un modelo generaliza bien cuando las prestaciones que se obtienen en el conjunto de test no son muy diferentes de las que se obtienen en el conjunto de entrenamiento.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />



En cualquier caso no conviene perder de vista que:
- **el conjunto de test sólo es un conjunto más**, es decir, que cuando eventualmente pongamos el modelo **en producción** vamos a tener que seguir monitorizando la generalización con los conjuntos de datos de test que estemos recibiendo (cada día, cada hora, cada semana, etc)
- A veces cuando refinamos un modelo **tomamos decisiones de diseño en función de las prestaciones que se alcanzan en el conjunto de test**. Esta práctica demasiado habitual introduce sesgos en la estimación de las prestaciones reales del modelo porque estamos **realimentando información del test** al entrenamiento del modelo. De algún modo es como si el test estuviese participando del entrenamiento, luego no se puede asegurar 100% que el conjunto de test sea independiente.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


En algunas bases de datos esta división viene hecha o sale de un modo natural, por ejemplo si las observaciones siguen un orden temporal (entrenar con datos obtenidos hasta una fecha y hacer el test con los datos obtenidos a partir de esa fecha). 

Para las situaciones en las que la división no viene hecha lo más habitual suele ser dejar más datos para entrenar que para testear, ya que el modelo que vaya a producción se va a entrenar con todos los datos disponibles:
- 50% entrenamiento, 50% test
- 70% entrenamiento, 30% test
- 80% entrenamiento, 20% test

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />



Existen maneras más fiables de estimar la capacidad de generalización que el empleo de un único conjunto de test. La más utilizada en la práctica es **validación cruzada**, que revisaremos en la próxima sesión.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, # observaciones
                                                    Y, # targets alineados con las observaciones
                                                    test_size=0.3, # ratio de observaciones que se van al conjunto de test,
                                                    random_state=42) # estado del generador de números aleatorios

## Entrenamiento y validación del modelo
Vamos a emplear la implementación de [*kNN*](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) contenida en el módulo [**sklearn**](https://scikit-learn.org/stable/) de python. El módulo sklearn contiene implementaciones de una gran variedad de algoritmos de aprendizaje automático, y la manera de consumirlas es la misma.
Todas las implementaciones tienen tres métodos fundamentales:
- `fit`: Entrenamiento del modelo. En los algoritmos de aprendizaje supervisado los argumentos son un array de `numpy` con las observaciones, una observación en cada **fila** y un array con los targets
- `predict`: Explotar el modelo. El argumento es un `numpy array` con las observaciones del conjunto de test. La salida es el target estimado para cada fila del argumento de entrada
- `score`: Evaluar el modelo. Cada modelo tiene una función de *score* particular, que es el estadístico que se calcula sobre las  predicciones hechas para el conjunto de test y nos cuantifica la calidad de estas predicciones. En un problema de clasificación se emplea la **tasa de aciertos**, es decir la fracción de las observaciones de test para las que el modelo ha encontrado la predicción correcta. Los argumentos para invocar el método `score` son las observaciones de test y los target verdaderos de test, a fin de poder medir estadísticos sobre las predicciones hechas por el modelo y estos *targets* reales.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=n_neighbors) # instanciar el objeto con hiperparámetros
knn.fit(x_train, y_train) # entrenar
acierto_entrenamiento = knn.score(x_train, y_train)
acierto_test = knn.score(x_test, y_test) # evaluar el modelo entrenado
print("Acierto en el conjunto de entrenamiento: {0:.2f}%".format(acierto_entrenamiento*100.))
print("Acierto en el conjunto de test: {0:.2f}%".format(acierto_test*100.))

In [ ]:
knn.predict(x_train)

### Primera valoración de los resultados
En primer lugar tenemos que comparar los resultados con un *baseline* trivial. Sería un valor que conseguiríamos sin ningún esfuerzo (sin entrenar). En este caso hay un clasificador que nos garantiza un acierto de en torno al 60%: decir que todos los correos son **no spam**

In [ ]:
baseline = max((1-np.mean(y_test)), np.mean(y_test))
baseline_spam = baseline
print("Acierto del baseline en el conjunto de test: {0:.2f}%".format(baseline*100.))

## Disección de los resultados
Vamos a repasar las condiciones en las que hemos hecho el entrenamiento del modelo y a valorar el impacto de cada una de las decisiones que podemos cambiar para condicionar el entrenamiento


### Impacto del número de vecinos


In [ ]:
v_nn = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
ac_entr = np.empty(len(v_nn))
ac_test = np.empty(len(v_nn))
for inn, n_neighbors in enumerate(v_nn):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(x_train, y_train)
    ac_entr[inn] = knn.score(x_train, y_train)
    ac_test[inn] = knn.score(x_test, y_test)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, ac_entr*100, linewidth=2, label='acierto entrenamiento')
plt.plot(v_nn, ac_test*100, linewidth=2,label='acierto test')
plt.plot(v_nn, np.ones(len(v_nn)) * baseline * 100., linewidth=2,label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()

In [ ]:
best_k = v_nn[np.argmax(ac_test)]
print("El numero de vecinos que da el mejor resultado en test es {0:d}, acierto del  {1:.2f}%".format(best_k, 
                                                                                                     100.*np.max(ac_test)))
acierto_spam_knn = 100.*np.max(ac_test)

### Ponderar el voto de cada vecino por su distancia

In [ ]:
##
##
##
##
##
ac_entr_w = np.empty(len(v_nn))
ac_test_w = np.empty(len(v_nn))
for inn, n_neighbors in enumerate(v_nn):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors,
                              weights='distance')
    knn.fit(x_train, y_train)
    ac_entr_w[inn] = knn.score(x_train, y_train)
    ac_test_w[inn] = knn.score(x_test, y_test)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, ac_entr_w*100, linewidth=2, label='acierto entrenamiento')
plt.plot(v_nn, ac_test_w*100, linewidth=2,label='acierto test')
plt.plot(v_nn, np.ones(len(v_nn)) * baseline* 100., linewidth=2,label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()

In [ ]:
best_k = v_nn[np.argmax(ac_test_w)]
print("El número de vecinos que da el mejor resultado en test si ponderamos por distancias es {0:d}, acierto del  {1:.2f}%".format(best_k, 
                                                                                                     100.*np.max(ac_test_w)))
acierto_spam_knn_w = 100.*np.max(ac_test_w)


### Bolas o vecinos

Los modelos basados en vecinos asumen que la **densidad** de observaciones es más o menos uniforme en todo el espacio donde residen las observaciones. En problemas con densidades muy variables, las distancias entre cada observación de test sus vecinos correspondientes pueden ser muy diversas con lo que cada decisión tiene en cuenta un concepto de "localización" diferente.

El siguiente código calcula la clasificación kNN con voto ponderado para todo el conjunto de test y para cualquier valor de $k$ desde 1 hasta el tamaño del conjunto de entrenamiento. Este código sólo funciona para valores razonablemente pequeños de tamaños de conjunto de entrenamiento. Saber desarrollar este código no es objetivo de este curso, está aquí para justificar intuitivamente cómo podemos encontrar pistas para intentar mejorar las prestaciones del kNN.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


Podemos ilustrar ese efecto con el *toy problem* eligiendo una muestra de test más aislada.

In [ ]:
xt = np.array([-0.2, -1])
fx,ax = plt.subplots(1,1,figsize=(7,7))
pinta_ejemplo(x,y,xt,ax)

In [ ]:
distancias = pairwise_distances(xt.reshape(1,-1), x,'euclidean') 

# Ordenar todos los vecinos para cada observación de test
id_vecinos = np.argsort(distancias) # indices de las observaciones de entrenamiento, ¿quién es el veino k-ésimo?

sorted_distancias = 1e-6+np.sort(distancias) # distancias a los vecinos ¿qué tan lejos está el vecino k-ésimo?

# clase a la que vota cada vecino
clase_vecinos = y[id_vecinos]
# peso con el que vota cada vecino a cada observación de test
peso_vecinos = 1./sorted_distancias
v_k = np.array([1,2,4,8])
ii=7
fx,ax = plt.subplots(len(v_k),2,figsize=(2*ii,len(v_k)*ii))
for ik,kk in enumerate(v_k):
  pinta_ejemplo(x,y,xt,ax[ik][0]) 
  pinta_radios(xt,np.array([x[cc,:] for cc in id_vecinos[0,:kk]]),ax[ik][0])
  pinta_votos(clase_vecinos[0,:kk], peso_vecinos[0,:kk]/np.sum(peso_vecinos[0,:kk]), ax[ik][1])

Volvemos al problema de detección de spam

In [ ]:
# calcular todos los pares de distancias entre observaciones de entenamiento y de test

A = pairwise_distances(x_test, x_train,'euclidean') 

# Ordenar todos los vecinos para cada observación de test
id_vecinos = np.argsort(A, axis=1) # indices de las observaciones de entrenamiento, ¿quién es el veino k-ésimo?
sorted_A = 1e-6+np.sort(A, axis=1) # distancias a los vecinos ¿qué tan lejos está el vecino k-ésimo?

# clase a la que vota cada vecino
clase_vecinos = np.vstack([y_train[id_vecinos[tt,:]] for tt in range(len(y_test))])

# peso con el que vota cada vecino a cada observación de test
peso_vecinos = 1./sorted_A

# denominadores para normalizar las sumas de votos
cumsum_peso_vecinos = np.cumsum(peso_vecinos, axis=1)

# resultados de las votaciones para cada observación de test y valor de k
knn_votos = np.cumsum(clase_vecinos * peso_vecinos, axis=1) / cumsum_peso_vecinos

# traducción de las votaciones en clasificación. En caso de empate (votación da 0.5) se resuelve hacia la clase 1
knn_clasificacion = np.zeros(A.shape, dtype=int)
knn_clasificacion[knn_votos >= 0.5] = 1

La siguiente figura representa la calidad de las predicciones de *kNN* en función de cómo de cerca estén los $k$ vecinos de la observación de test correspondiente. Los radios de las vecindades se han discretizado en 50 bins con escala logarítmica. Cada bin de radios discretizados se representa con un círculo en el la columna de la derecha. El tamaño del círculo es proporcional al número de observaciones del conjunto de test que se han clasificado con un radio de vecindad que cae dentro del bin (este número es la altura del bin en el histograma de la izquierda).

Vemos claramente cómo al aumentar el radio de la vecindad, para un mismo número de vecinos, las prestaciones del *kNN* se degradan porque ciertamente se diluye la asunción de que los vecinos están cerca de la observación de test.

In [ ]:
v_k=[1, 3, 5, 10]
ff,aa = plt.subplots(len(v_k), 2, sharex='row', sharey='col', figsize=(10,12))
bins=50
for ik, k in enumerate(v_k):
    radios = sorted_A[:,k]
    y_pred = knn_clasificacion[:,k]
    acierto = y_pred==y_test
    logbins = np.logspace(0,np.log10(np.max(radios)+1),(bins))
    aa[ik][0].hist(radios, logbins)
    aa[ik][0].set_xscale('log')
    aa[ik][0].set_title('histograma de radios de vecindad, k={0:d}'.format(k))
    
    id_bin = np.digitize(radios, logbins)
    prob_acierto_radio = np.empty(len(logbins))
    tamanno = np.empty(bins)
    for ii in range(bins):
        cuales = np.where(id_bin == ii)[0]
        if len(cuales)<2:
            continue
        tamanno[ii] = len(cuales)
        prob_acierto_radio[ii] = np.mean(acierto[cuales])
    
    aa[ik][1].scatter(logbins, prob_acierto_radio,s=10*tamanno, alpha=0.3)
    aa[ik][1].set_title('Probabilidad de acertar')
    aa[ik][1].set_xlabel('radio de vecindad')
    aa[ik][1].grid
ff.tight_layout()

En sklearn disponemos de una alternativa a *kNN* que nos permite definir la vecindad de cada observación de test en función de un radio: `sklearn.neighbors.RadiusNeighborsClassifier`. Existen dos diferencias fundamentales con respecto a *kNN*:
1. El radio no te garantiza que la vecindad no esté vacía: `outlier_label` se encarga de gestionar a esas observaciones tristes y solitarias
2. El tiempo de ejecución es sensiblemente mayor para RadiusNeighborsClassifier

In [ ]:
##
## 
##
##
##
##
from sklearn.neighbors import RadiusNeighborsClassifier
rad_ac_entr = np.empty(bins)
rad_ac_test = np.empty(bins)
for ir, radius in enumerate(logbins):
    rknn = RadiusNeighborsClassifier(radius=radius,  outlier_label='most_frequent')
    rknn.fit(x_train, y_train)
    rad_ac_entr[ir] = rknn.score(x_train, y_train)
    rad_ac_test[ir] = rknn.score(x_test, y_test)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(logbins, rad_ac_entr*100, lw=2, label='acierto entrenamiento')
plt.plot(logbins, rad_ac_test*100, lw=2,label='acierto test')
plt.plot(logbins, np.ones(bins) * baseline, lw=2,label='baseline')
plt.xscale('log')
_ = plt.xlabel('Radio vecindad')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()

In [ ]:
##
## 
##
## 
##
##
from sklearn.neighbors import RadiusNeighborsClassifier
rad_ac_entr_w = np.empty(bins)
rad_ac_test_w = np.empty(bins)
for ir, radius in enumerate(logbins):
    wrknn = RadiusNeighborsClassifier(radius=radius,  outlier_label='most_frequent',weights='distance')
    wrknn.fit(x_train, y_train)
    rad_ac_entr_w[ir] = wrknn.score(x_train, y_train)
    rad_ac_test_w[ir] = wrknn.score(x_test, y_test)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(logbins, rad_ac_entr_w*100, lw=2, label='acierto entrenamiento')
plt.plot(logbins, rad_ac_test_w*100, lw=2, label='acierto test')
plt.plot(logbins, np.ones(bins) * baseline, lw=2, label='baseline')
plt.xscale('log')
_ = plt.xlabel('Radio vecindad')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()

### ¿Cuál ha sido la mejor estrategia para este problema?
Pues parece que si miramos el acierto de clasificación en el conjunto de test, la mejor estrategia haya sido *5NN* con voto ponderado por distancia.

# Regresión con $k$NN
## Funciones de pérdidas y riesgos 

Las **funciones de pérdidas** miden el coste del error obtenido al predecir el *target* para una observación. El error se define como la diferencia entre el valor real del *target* ($y$) y el estimado ($\hat{y}=f(\mathbf x)$). Las funciones de pérdidas más habituales son

- Coste cuadrático
$$
l(y, \hat{y}) = (y - \hat{y})^2
$$
- Error absoluto
$$
l(y, \hat{y}) = |y - \hat{y}|
$$

Cualitativamente ambas funciones de pérdida se parecen en que la penalización por acertar es nula, pero se diferencian en el trato que hacen sobre los *outliers*, las penalizaciones por errores muy gruesos son mucho mayores con el coste cuadrático, por lo que los algoritmos que traten de optimizar esta función de pérdidas harán mayor esfuerzo en tener pocas estimaciones con errores muy gruesos.

El **riesgo** de un modelo de regresión es el valor esperado de su función de pérdidas. Es una manera de medir la calidad de las estimaciones hechas con el modelo. 
- El riesgo correspondiente a usar coste cuadrático como función de pérdidas es el **error cuadrático medio** (MSE, *mean squared error*)
- El riesgo correspondiente al error absoluto es el **error absoluto medio** (MAD, *mean absolute deviation*, MAE, *mean absolute error*)



## 1. Estimación del *target* $y_t$ para una observación de test $\mathbf x_t$
- Buscar de entre las observaciones del conjunto de entrenamiento las $k$ que están a una **distancia** menor de $\mathbf x_t$: Estas muestras son los $k$ vecinos más próximos de $\mathbf x_t$
- Rescatar los *targets* de cada uno de los $k$ vecinos. 
- Asignar como *target* a la observación de test, $y_t$:
 - La **media** de los *targets* de los vecinos
 - La **media** de los *targets* de los vecinos, ponderados por la distancia entre vecino y observación de test
 - La **mediana** de los targets de los vecinos
 - La **moda** de los targets de los vecinos
 
 El estadístico empleado depende de la **función de coste** que queramos optimizar mediante el modelo:
 - Si queremos optimizar el MSE, el estadístico que debemos usar es la **media** de los *targets*. Esto es así porque la media es sensible a los *outliers* (todo suma).
 - Si queremos optimizar el MAE, debemos usar la **mediana** de los targets. La mediana es más difícil de influenciar por outliers (depende del orden de los targets
 - En casos donde los targets sean discretos (un conjunto más o menos reducido de valores), la **moda** puede ser una buena opción para no asignar valores de *target* fuera del dominio.

# Problema de regresión para esta sesión: Housing 
Es otra de las bases de datos clásicas de UCI. Se trata de estimar el precio de propiedades en California a partir de algunas características de las mismas. Esta base de datos [**se puede cargar directamente desde sklearn**]



  

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
nn_california = 5000
data = housing.data[:nn_california]
targets = housing.target[:nn_california]
data_df = pd.DataFrame(np.vstack((data.T,targets)).T, columns=housing['feature_names']+ housing['target_names'])

In [ ]:
print(housing.DESCR)

In [ ]:
data_df.head()

In [ ]:
data_df.describe()

## Separar entre entrenamiento y test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.3, random_state=42)

## Entrenamiento y validación del modelo
Vamos a emplear la implementación de [*kNN* para regresión](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) contenida en el módulo **sklearn** de python.
Los regresores implementados en sklearn también tienen los tres métodos fundamentales que vimos para clasificación:
- `fit`: Entrenamiento del modelo. En los algoritmos de aprendizaje supervisado los argumentos son un array de numpy con las observaciones, una observación en cada **fila** y un array con los targets
- `predict`: Explotar el modelo. El argumento es un numpy array con las observaciones del conjunto de test. La salida es el target estimado para cada fila del argumento de entrada
- `score`: Evaluar el modelo. En los problemas de regresión el *score* que se calcula por defecto es el estadístico $R^2$.


### El estadístico $R^2$

$$
R^2 = 1-\frac{\sum_{i=1}^{N_t}{(y_i-\hat{y}_i)^2}}{\sum_{i=1}^{N_t}{(y_i-\bar{y})^2}}
$$donde $N_t$ es el tamaño del conjunto usado para calcular el estadístico, $\hat{y}_i$, $i=1,\dots,N_t$ son las estimaciones hechas por el modelo, $y_i$ ($i=1,\dots,N_t$)  son los *targets* verdaderos e $\bar{y}=1/N_t\sum_{i=1}^{N_t}y_i$ es la media de los *targets* verdaderos. 

- $R^2$ toma un valor máximo de $1.0$ cuando el modelo no comete errores y su valor es decreciente. 

- La manera que tiene de decrecer es una comparación entre la precisión del modelo y la precisión de un modelo trivial consistente en estimar todos los *targets* con la media. 

- Mientras el modelo sea más preciso que estimar con la media, $R^2$ será positivo. Si obtenemos un valor de $R^2$ negativo es porque el modelo comete un error peor que el resultante de estimar todos los targets con su media. 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

n_neighbors = 3
knn = KNeighborsRegressor(n_neighbors=n_neighbors)
knn.fit(x_train, y_train)
riesgo_entrenamiento = knn.score(x_train, y_train)
riego_test = knn.score(x_test, y_test)
print("R^2 en el conjunto de entrenamiento: {0:.2f}".format(riesgo_entrenamiento))
print("R^2 en el conjunto de test: {0:.2f}".format(riego_test))

### Primera valoración de los resultados
Hemos obtenido un $R^2$ negativo y razonablemente cerca de 0. 

### Impacto del número de vecinos

In [ ]:
##
## 
##
##
##
##
from sklearn.metrics import r2_score
v_nn = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,350]
r2_entr = np.empty(len(v_nn))
r2_test = np.empty(len(v_nn))
for inn, n_neighbors in enumerate(v_nn):
    knn = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn.fit(x_train, y_train)
    r2_entr[inn] = knn.score(x_train, y_train)
    r2_test[inn] = knn.score(x_test, y_test)


In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, r2_entr, lw=2,label='$R^2$ entrenamiento')
plt.plot(v_nn, r2_test, lw=2,label='$R^2$ test')
plt.plot(v_nn, r2_score(y_test,np.ones(len(y_test))*np.mean(y_train))*np.ones(len(v_nn)), lw=2, label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('$R^2$')
_ = plt.legend()

plt.grid()
best_k = v_nn[np.argmax(r2_test)]
plt.show()
print("El número de vecinos que da el mejor resultado en test es {0:d}, R^2 del  {1:.2f}".format(best_k, 
                                                                                                     np.max(r2_test)))
acierto_housing_knn = np.max(r2_test)

En escala logarítmica, para ver en mejor detalle los números bajos de $k$

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, r2_entr, lw=2,label='$R^2$ entrenamiento')
plt.plot(v_nn, r2_test, lw=2,label='$R^2$ test')
plt.plot(v_nn, r2_score(y_test,np.ones(len(y_test))*np.mean(y_train))*np.ones(len(v_nn)), lw=2, label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('$R^2$')
_ = plt.legend()
plt.xscale('log')
plt.grid()

In [ ]:
##
## 
##
##
##
##
r2_entr_w = np.empty(len(v_nn))
r2_test_w = np.empty(len(v_nn))
for inn, n_neighbors in enumerate(v_nn):
    knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights='distance')
    knn.fit(x_train, y_train)
    r2_entr_w[inn] = knn.score(x_train, y_train)
    r2_test_w[inn] = knn.score(x_test, y_test)
              

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, r2_entr_w, lw=2,label='$R^2$ entrenamiento')
plt.plot(v_nn, r2_test_w, lw=2,label='$R^2$ test')
plt.plot(v_nn, r2_score(y_test,np.ones(len(y_test))*np.mean(y_train))*np.ones(len(v_nn)),
         lw=2,label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('$R^2$')
_ = plt.legend()
plt.grid()
best_k = v_nn[np.argmax(r2_test_w)]
plt.show()
print("El número de vecinos que da el mejor resultado si ponderamos votos por distancias en test es {0:d}, R^2 del  {1:.2f}".format(best_k,np.max(r2_test_w))) 

acierto_housing_knn_w = np.max(r2_test_w)   

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(v_nn, r2_entr_w, lw=2, label='$R^2$ entrenamiento')
plt.plot(v_nn, r2_test_w, lw=2, label='$R^2$ test')
plt.plot(v_nn, r2_score(y_test,np.ones(len(y_test))*np.mean(y_train))*np.ones(len(v_nn)), 
         lw=2, label='baseline')
_ = plt.xlabel('Num vecinos')
_ = plt.ylabel('$R^2$')
_ = plt.legend()
plt.xscale('log')
plt.grid()

### Bolas o vecinos

In [ ]:
A = pairwise_distances(x_test, x_train,'euclidean') 

# Ordenar todos los vecinos para cada observación de test
id_vecinos = np.argsort(A, axis=1) # indices de las observaciones de entrenamiento, ¿quién es el veino k-ésimo?
sorted_A = 1e-6+np.sort(A, axis=1) # distancias a los vecinos ¿qué tan lejos está el vecino k-ésimo?

# target que vota cada vecino
target_vecinos = np.vstack([y_train[id_vecinos[tt,:]] for tt in range(len(y_test))])

# peso con el que vota cada vecino a cada observación de test
peso_vecinos = 1./sorted_A

# denominadores para normalizar las sumas de votos
cumsum_peso_vecinos = np.cumsum(peso_vecinos, axis=1)

# resultados de las votaciones para cada observación de test y valor de k
knn_estimacion = np.cumsum(target_vecinos * peso_vecinos, axis=1) / cumsum_peso_vecinos



In [ ]:
v_k=[1, 3, 5, 10]
ff,aa = plt.subplots(len(v_k), 2, sharex='row', sharey='col', figsize=(8,12))
bins=20
for ik, k in enumerate(v_k):
    radios = sorted_A[:,k]
    y_pred = knn_estimacion[:,k]
    #acierto = 
    logbins = np.logspace(0,np.log10(np.max(radios)+1),(bins))
    aa[ik][0].hist(radios, logbins)
    aa[ik][0].set_xscale('log')
    aa[ik][0].set_title('histograma de radios de vecindad, k={0:d}'.format(k))
    
    id_bin = np.digitize(radios, logbins)
    prob_acierto_radio = np.empty(len(logbins))
    tamanno = np.empty(bins)
    for ii in range(bins):
        cuales = np.where(id_bin == ii)[0]
        if len(cuales)<2:
            continue
        tamanno[ii] = len(cuales)
        prob_acierto_radio[ii] = r2_score(y_test[cuales], y_pred[cuales])
    
    aa[ik][1].scatter(logbins, prob_acierto_radio,s=10*tamanno, alpha=0.3)
    aa[ik][1].set_title('R^2 medio')
    aa[ik][1].set_xlabel('radio de vecindad')
    aa[ik][1].grid
ff.tight_layout()


Lamentablemente **no podemos usar** `RadiusNeighborsRegressor` porque no tiene el control de vecindarios vacíos que tiene la versión para clasificación...

Otras funcionalidades que estaría bien añadir al regresor es que el *target* fuese la mediana o la moda de los *targets* de los vecinos...

# Árboles de Decisión
Construyen una estructura jerárquica mediante la aplicación **recursiva** de clasificadores de tipo *stump*

## Clasificador *stump*
- Definido mediante una variable y un umbral.
- Divide el espacio de entrada en dos, aquellos patrones para los que el test es verdadero y aquellos patrones para los que el test es falso.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Construcción del árbol
1. Se comienza con un **nodo raíz**, que es un nodo que contiene a todas las observaciones del conjunto de entrenamiento
2.  Se determina un test de umbral para el nodo raíz que divide este nodo de dos **nodos rama**
3.  **Recursión** aplicada a cada nodo rama
    1. Si el nodo rama es **suficientemente homogéneo** se declara **nodo hoja**
        - se le asigna una clase de salida para etiquetar observaciones
        - y se saca de la lista de nodos rama
    2. Si no es suficientemente homogéneo se subdivide a su vez en dos nodos rama; el nodo que se acaba de subdividir se saca de la lista de nodos hoja
    
<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Evaluación del árbol
1. Cada observación de test se evalúa con el test de umbral del nodo raíz e ingresa en una de las ramas del árbol
2. Recursivamente va descendiendo a través de los test de umbral de los nodos rama que le toque recorrer
3. Se asigna la clase target del nodo hoja en el que termine su recorrido por la jerarquía del árbol

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Optimización de cada test de umbral
Si a un nodo $h_0$ llegan $N$ observaciones en $d$ dimensiones (esto es, $d$ variables de entrada), cada test se optimiza mediante un bucle anidado que se ejecuta por cada dimensión $j=1,\dots,d$:

Repetir para $j=1,\dots,d$:  
1. Ordenar las observaciones por sus valores de la variable $j$. Si asumimos que los datos que llegan al nodo están ordenados en un array de $N$ filas (una por cada dato) y $d$ columnas (una por cada dimensión), este paso consiste en ordenar las observaciones por el valor de la columna $j$. 
2. Existen como mucho $N-1$ posibles test de umbral
  - Hay $N-1$ maneras de separar las observaciones ordenadas por la dimensión $j$ en dos grupos
  - si hay valores repetidos para una variable habrá menos de $N-1$ posibles umbrales
3. Repetir para cada uno de los $N-1$ umbrales en la dimensión $j$:  
    - Suponemos el test de umbral divide el nodo rama en dos nodos hijos, $h_1$ y $h_2$, donde
        - $h_1$ recibe $N_1$ de las $N$ observaciones (aquellas cuyo valor de la variable $j$ está a la **izquierda** del umbral), la clase mayoritaria es $t_1$ y la impureza del nodo es $G_1$
        - $h_2$ recibe $N_2$ (aquellas cuyo valor de la variable $j$ está a la **derecha** del umbral) de las $N$ observaciones  ($N_1 + N_2=N$), la clase mayoritaria es $t_2$ y la impureza del nodo es $G_2$
    - Evaluamos y almacenamos la **mediocridad** del test como $\frac{N_1}{N}G_1 + \frac{N_2}{N}G_2$

Finalmente elegimos como test de umbral para dividir $h_0$ el mejor de los $(N-1)\times d$ test probados en el bucle anidado, es decir, el de **menor mediocridad**.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Medidas de mediocridad de test de umbral para clasificación
- Error de clasificación
- Índice **Gini**. Si tenemos un problema de clasificación con $K$ clases, definimos como $p_k$ la fracción de observaciones del nodo que pertenecen a la clase con target $t_k$
  $$
  \mbox{Gini}=\sum_{k=1}^Kp_k(1-p_k)
  $$
- **Entropía cruzada**
  $$
  -\sum_{k=1}^Kp_k\log p_k
  $$
La siguiente figura representa los valores de estas tres medidas en un problema con dos clases. Intuitivamente el índice Gini o la entropía se prefieren al error porque ante divisiones que dan el mismo error de clasificación, prefieren las que tienen a uno de los hijos más homogéneo.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
xx=np.linspace(1e-4,.9999,100)
error = np.min(np.vstack((xx,1-xx)),0)
gini = 2*xx*(1-xx)
entropia = -xx*np.log(xx)-(1-xx)*np.log(1-xx)
plt.figure()
plt.plot(xx,error,label='Error Clasificación')
plt.plot(xx,gini,label='Índice Gini')
plt.plot(xx,entropia,label='Entropía Cruzada')
plt.xlabel('p(y=1)')
plt.legend()
plt.grid()

### Ejemplo con el *toy problem*

In [ ]:
fx,ax = plt.subplots(1,1,figsize=(7,7))
pinta_ejemplo(x,y,xt,ax)
ax.set_xlabel('x1')
_ = ax.set_ylabel('x2')

In [ ]:
from sklearn.tree import DecisionTreeClassifier,export_text
DT = DecisionTreeClassifier(max_leaf_nodes=20,random_state=42)
DT.fit(x, y)

import graphviz
from sklearn import tree
dot_Data = tree.export_graphviz(DT, 
                                out_file=None, 
                                feature_names = ['x1','x2'],
                               class_names=['azul','roja'],
                                filled=True, 
                                rounded=True,  
                              special_characters=True)
graph = graphviz.Source(dot_Data)
graph

In [ ]:
r = export_text(DT, feature_names=['x1','x2'])
print(r)

In [ ]:
fx,ax = plt.subplots(5,1,sharex=True, sharey=True, figsize=(6,30))
cc=0
plot_step=0.05
x0 = x.copy()
x_min, x_max = x0[:, 0].min() - 0.1, x0[:, 0].max() + 0.1
y_min, y_max = x0[:, 1].min() - 0.1, x0[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
v_hh = [2,3,4,5,6]
y1 = y.copy()
y1[y1==0] = -1
for cc in range(5):
  DT = DecisionTreeClassifier(max_leaf_nodes=v_hh[cc],random_state=42)
  DT.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc])
  ax[cc].set_xlabel('x1')
  ax[cc].set_ylabel('x2')

  Z = DT.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc].set_ylim([y_min, y_max])


## Implementación en sklearn

La clase [`sklearn.tree.DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) tiene los siguientes parámetros:
- `criterion`: "gini" o "entropy"
- `splitter`: "best" o "random"
- `max_depth`: máxima profundidad
- `min_samples_split`: mínimo número de observaciones para dividir el nodo
- `min_samples_leaf`: mínimo número de observaciones para que un nodo pueda ser hoja
- `min_weight_fraction_leaf`: para cuando tenemos un peso para cada observación, mismo concepto que min_samples_leaf
- `max_features`: máximo número de variables en el bucle para buscar el mejor test de umbral
- `max_leaf_nodes`: número máximo de hojas en *best-first*
- `min_impurity_decrease`: un nodo se divide si consigue que la impureza decaiga al menos esta cantidad
- `min_impurity_split`: un nodo se divide sólo si su impureza pasa este valor. Si no lo pasa se queda en hoja
- `class_weight`: peso de los errores de cada clase

## Spam detection con árboles de decisión

In [ ]:
data = load_spam()
X = data[data.columns[:57]].values
Y = data['target'].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
baseline = max(100.*(1-np.mean(y_test)), 100.*np.mean(y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(max_leaf_nodes=6)
DT.fit(x_train, y_train)
acierto_entrenamiento = DT.score(x_train, y_train)
acierto_test = DT.score(x_test, y_test)
print("Acierto en el conjunto de entrenamiento: {0:.2f}%".format(acierto_entrenamiento*100.))
print("Acierto en el conjunto de test: {0:.2f}%".format(acierto_test*100.))

Los árboles de decisión (si no tienen muchos nodos) se pueden visualizar

In [ ]:
import graphviz
from sklearn import tree
dot_Data = tree.export_graphviz(DT, 
                                out_file=None, 
                                feature_names = data.columns[:-1],
                               class_names=['no spam','spam'],
                                filled=True, 
                                rounded=True,  
                              special_characters=True)
graph = graphviz.Source(dot_Data)
graph

### Influencia del número de hojas

In [ ]:
##
##
##
##
##
##
v_num_hojas = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

dt_entr = np.empty(len(v_num_hojas))
dt_test = np.empty(len(v_num_hojas))
for ih,hh in enumerate(v_num_hojas):
    DT = DecisionTreeClassifier(max_leaf_nodes=hh)
    DT.fit(x_train, y_train)
    dt_entr[ih] = 100.*DT.score(x_train, y_train)
    dt_test[ih] = 100.*DT.score(x_test, y_test)
plt.figure()
plt.plot(v_num_hojas, dt_entr, label='Acierto entrenamiento')
plt.plot(v_num_hojas, dt_test, label='Acierto test')
plt.plot(v_num_hojas, baseline*np.ones(len(v_num_hojas)), label='baseline')
_ = plt.xlabel('Num hojas')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()
best_k = v_num_hojas[np.argmax(dt_test)]
plt.show()
print("El número de hojas que da el mejor resultado en test es {0:d}, acierto del  {1:.2f}".format(best_k, 
                                                                                                     np.max(dt_test)))
acierto_spam_dt = np.max(dt_test)

# Parámetros entrenables e hiperparámetros

Dentro del entrenamiento árbol de decisión hay que asignar valores a dos tipos de parámetros:
- **Parámetros** entrenables: Variable y umbral en cada split. Se ajustan durante la optimización que se produce al invocar el método `fit`
- **Hiperparámetros**: se ajustan durante la creación del árbol (llamada al método `__init__` que se hace al ejecutar `DecisionTreeClassifier()`.  Condicionan la optimización del método `fit`. Usamos información a priori para ajustarlos, es la manera que tenemos de controlar la **capacidad de generalización del árbol**.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


# Sobreajuste 

Cada modelo de aprendizaje automático lleva implícita una **capacidad expresiva**: potencialidad de ajustar los valores de sus parámetros o su arquitectura para capturar patrones en los datos. En general, cuantos más parámetros (o más compleja sea la estructura), mayor capacidad expresiva. 

En el árbol de decisión el número de nodos marca la capacidad expresiva.

La esencia del entrenamiento del modelo consiste en capturar patrones existentes en los datos debidos a la tarea que queremos resolver. Una manera trivial de resolver el problema de spam para el conjunto de entrenamiento es construir un árbol donde cada observación acabe en una hoja independiente (equivalente a un clasificador 1NN). El acierto de este modelo superexpresivo en el conjunto de entrenamiento será del 100%. Pero no conseguimos un acierto del 100% en el conjunto de test: estamos **sobreajustando** (*overfitting*) los parámetros del modelo al conjunto de entrenamiento.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />



Si una hoja del árbol contiene una única observación, esa observación no ha llegado a esa hoja por ser spam o no spam, ha llegado ahí por ser diferente a todas las demás observaciones. Por lo tanto esa rama no está capturando un patrón que nos ayude a separar spam de no spam, sino a separar correos idénticos a la observación que define la hoja de cualquier otro correo, independientemente de su clase *target*.

Limitando el número de hojas podemos controlar el sobreajuste pero corremos el riesgo de **limitar la capacidad expresiva** impidiendo que el modelo pueda capturar todos los patrones relevantes.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [ ]:
v_num_hojas = [2,5,10, 20, 200, 2000]

dt_entr = np.empty(len(v_num_hojas))
dt_test = np.empty(len(v_num_hojas))
for ih,hh in enumerate(v_num_hojas):
    DT = DecisionTreeClassifier(max_leaf_nodes=hh)
    DT.fit(x_train, y_train)
    dt_entr[ih] = 100.*DT.score(x_train, y_train)
    dt_test[ih] = 100.*DT.score(x_test, y_test)
plt.figure()
plt.plot(v_num_hojas, dt_entr, label='Acierto entrenamiento')
plt.plot(v_num_hojas, dt_test, label='Acierto test')
plt.plot(v_num_hojas, baseline*np.ones(len(v_num_hojas)), label='baseline')
plt.xscale('log')
plt.fill_between(np.array([200,2000]),
                dt_entr[-2:],
                dt_test[-2:],
                color='red',
                alpha=0.2,
                label='sobreajuste')
plt.fill_between(np.array([2,5]),
                dt_entr[:2],
                dt_test[:2],
                color='olive',
                alpha=0.6,
                label='subajuste')
_ = plt.xlabel('Num hojas')
_ = plt.ylabel('Acierto (%)')
_ = plt.legend()
plt.grid()


## Ejercicio
Estudiar la influencia de alguno de los **hiperparámetros** de los que depende la configuración del árbol de decisión. Reemplazando `max_leaf_nodes` por otro parámetro intentar conseguir árboles parecidos.

# Árboles de Regresión

La estructura jerárquica del árbol de decisión se puede adaptar fácilmente a problemas de regresión cambiando la función con la que se evalúa la mediocridad de cada test de umbral.

## Medidas de mediocridad de test de umbral para regresión
- Error cuadrático medio (MSE). Si empleamos esta medida de mediocridad, el *target* asignado al nodo (equivalente de la clase mayoritaria para clasificación) debe ser la **media** de los targets de las observaciones del conjunto de entrenamiento que llegan al nodo.
- Error absoluto medio (MAE). Si empleamos esta medida de mediocridad, el *target* asignado al nodo debe ser la **mediana**.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />



## Optimización de cada test de umbral para regresión
Si a un nodo $h_0$ llegan $N$ observaciones en $d$ dimensiones, cada test se optimiza mediante un bucle anidado que se ejecuta por cada dimensión:
1. Ordenar las observaciones por sus valores de la variable $(\mathbf x)_j$, $j=1,\dots,d$
2. Existen como mucho $N-1$ posibles test de umbral (si hay valores repetidos para una variable habrá menos de $N-1$ posibles umbrales)
3. Bucle que recorre cada uno de los $N-1$ umbrales
    - Suponemos el test de umbral divide el nodo rama en dos nodos hijos, $h_1$ y $h_2$, donde
        - $h_1$ recibe $N_1$ de las $N$ observaciones y $h_2$ recibe $N_2$ de las $N$ observaciones  ($N_1 + N_2=N$),
        - Si se optimiza el MSE:
            - el target asignado al nodo $h_1$ será la media de las $N_1$ observaciones que le correspondan (idem para  $h_2$)
            - la mediocridad del nodo hijo $G_1$ será el MSE resultante de estimar las $N_1$ observaciones que caen en $h_1$ con su media (idem para  $h_2$)
        - Si se optimiza el MAE:
            - el target asignado al nodo $h_1$ será la mediana de las $N_1$ observaciones que le correspondan (idem para  $h_2$)
            - la mediocridad del nodo hijo $G_1$ será el MAE resultante de estimar las $N_1$ observaciones que caen en $h_1$ con su mediana (idem para  $h_2$)
    - Evaluamos la **mediocridad** del test como $\frac{N_1}{N}G_1 + \frac{N_2}{N}G_2$
4. Elegimos como test de umbral para dividir $h_0$ el mejor de los $(N-1)\times d$ test probados en el bucle anidado

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


# Housing con árboles de regresión

In [ ]:
housing = fetch_california_housing()
data = housing.data[:nn_california]
targets = housing.target[:nn_california]
data_df = pd.DataFrame(np.vstack((data.T,targets)).T, columns=housing['feature_names']+ housing['target_names'])
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor(max_leaf_nodes=8)
DT.fit(x_train, y_train)
riesgo_entrenamiento = DT.score(x_train, y_train)
riego_test = DT.score(x_test, y_test)
print("R^2 en el conjunto de entrenamiento: {0:.2f}".format(riesgo_entrenamiento))
print("R^2 en el conjunto de test: {0:.2f}".format(riego_test))

In [ ]:
dot_Data = tree.export_graphviz(DT, 
                                out_file=None, 
                                feature_names = housing['feature_names'],
                                filled=True, 
                                rounded=True,  
                              special_characters=True)
graph = graphviz.Source(dot_Data)
graph

In [ ]:
##
##
##
##
##
##
v_num_hojas = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100, 300]

dt_entr = np.empty(len(v_num_hojas))
dt_test = np.empty(len(v_num_hojas))
for inn, hh in enumerate(v_num_hojas):
    DT = DecisionTreeRegressor(max_leaf_nodes=hh)
    DT.fit(x_train, y_train)
    dt_entr[inn] = DT.score(x_train, y_train)
    dt_test[inn] = DT.score(x_test, y_test)
plt.figure()
plt.plot(v_num_hojas, dt_entr, label='$R^2$ entrenamiento')
plt.plot(v_num_hojas, dt_test, label='$R^2$ test')
plt.plot(v_num_hojas, r2_score(y_test,np.ones(len(y_test))*np.mean(y_train))*np.ones(len(v_num_hojas)), label='baseline')
_ = plt.xlabel('Num hojas')
_ = plt.ylabel('$R^2$')
_ = plt.legend()
plt.grid()
plt.xscale('log')
best_k = v_nn[np.argmax(dt_test)]
plt.show()
print("El número de hojas que da el mejor resultado en test es {0:d}, R^2 del  {1:.2f}".format(best_k, 
                                                                                                     np.max(dt_test)))
acierto_housing_dt = np.max(dt_test)

## Ventajas y limitaciones de árboles
- :o) Resuelven de modo natural problemas de regresión, clasificación binaria y multiclase
- :o) Los splits binarios (aún en problemas multiclase, o con variables categóricas) ayudan a controlar el compromiso entre generalización y capacidad expresiva
- :o) Son fáciles de explicar (que no de interpretar, salvo que haya pocos nodos)
- :o) Son fáciles y relativamente poco costosos de evaluar
- :o( Fronteras de clasificación que no sean paralelas a los ejes son costosas de recrear
- :o( La optimización es de tipo *greedy*, los nodos rama más pegados a la raíz son determinantes en la solución global
- :o( Compromiso entre número de observaciones y número de variables. A medida que se avanza en la jerarquía cada vez hay menos observaciones para entrenar los nodos hoja. Si el número de variables es alto, podemos incurrir en sobreajustes (que el árbol separe observaciones individuales, no detecte patrones relacionados con la tarea a resolver)

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


# Random Forests

## Modelos basados en conjuntos (*ensemble methods*)
Hay una manera de robustecer modelos de aprendizaje automático que consiste en entrenar un conjunto de modelos con el mismo conjunto de entrenamiento de partida pero introduciendo elementos que introduzcan diversidad:
- submuestreando el conjunto de entrenamiento
- inicializando aleatoriamente alguno de los parámetros de diseño de cada miembro del conjunto.  

La manera de explotar el conjunto para hacer predicciones es que cada miembro hace su predicción para cada observación del conjunto de test y todas estas predicciones se combinan en una predicción global.
- Tomando la media de las predicciones
- Tomando la moda de las predicciones
- Ponderando los votos por la fiabilidad de cada miembro, etc.

De esta manera se consigue limitar la capacidad de sobreajuste o diluirla.

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Random Forest para clasificación o regresión

Es un algoritmo para generar $B$ árboles de decisión que se integran en un modelo de conjuntos orientado a resolver un problema de clasificación o regresión (recordad el algoritmo para generar cada árbol es el mismo sólo cambia la función de mediocridad y la manera de calcular el *target* que se asigna a cada nodo hijo.

El algoritmo es un bucle que se repite $B$ veces, una por cada árbol del bosque. Como entrada se reciben $N$ observaciones en $d$ dimensiones, con sus correspondientes *targets*, y un **tamaño de bosque** $B$.

1. Elegir al azar un subconjunto de observaciones del conjunto de entrenamiento de tamaño $N_b$
2. Aprender el árbol $T_b$ usando como conjunto de entrenamiento las $N_b$ observaciones pero introduciendo la siguiente modificación en el algoritmo del árbol:
   - Antes de aprender cada *stump* elegir al azar $m_b$ variables de las $d$ disponibles, es decir, cada nodo sólo ve un subconjunto de las variables de entrada para optimizar su test de umbral
3. Para hacer una predicción. Cada observación del conjunto de test se predice con los $B$ árboles. La predicción final del bosque es
    - en bosques de regresión: la media de las $B$ predicciones
    - en bosques de clasificación: la clase más votada
    
<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />



## Random Forest en sklearn para clasificación

El módulo que implementa *Random Forest* para clasificación es [`sklearn.ensemble.RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). La API tiene hiperparámetros para controlar la capacidad de generalización de cada árbol y la del bosque en sí. Algunos de estos hiperparámetros son:
- `n_estimators`: Tamaño del bosque
- `bootstrap`: si usar o no todas las observaciones con cada árbol
- `oob_score`: medida de relevancia de cada variable
- `max_samples`: si hay bootstrap, número de observaciones para entrenar cada árbol

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


## Random Forest y spam detection

In [ ]:
data = load_spam()
X = data[data.columns[:57]].values
Y = data['target'].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
baseline = max(100.*(1-np.mean(y_test)), 100.*np.mean(y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(max_leaf_nodes=6,
                           n_estimators=100)
RF.fit(x_train, y_train)
acierto_entrenamiento = RF.score(x_train, y_train)
acierto_test = RF.score(x_test, y_test)
print("Acierto en el conjunto de entrenamiento: {0:.2f}%".format(acierto_entrenamiento*100.))
print("Acierto en el conjunto de test: {0:.2f}%".format(acierto_test*100.))

### Influencia del tamaño del bosque y de la complejidad de cada árbol

In [ ]:

v_hojas = [2,3,4,5,10,20,50,100]
v_arboles = [5,10,20,50,100,200,500, 1000]



In [ ]:
ff,aa = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12,8))
best_test = 0
best_num_arboles = -1
best_num_hojas = -1
for kk, hh in enumerate(v_hojas): 
    dt_entr = np.empty(len(v_arboles))
    dt_test = np.empty(len(v_arboles))
    for ih,B in enumerate(v_arboles):
        RF = RandomForestClassifier(max_leaf_nodes=hh,
                       n_estimators=B)
        RF.fit(x_train, y_train)
        dt_entr[ih] = 100.*RF.score(x_train, y_train)
        dt_test[ih] = 100.*RF.score(x_test, y_test)
    if np.max(dt_test) > best_test:
      best_test = np.max(dt_test)
      best_num_arboles = B
      best_num_hojas = hh    
    aa[0].plot(v_arboles, dt_entr, label='{0:d} hojas'.format(hh))
    aa[1].plot(v_arboles, dt_test, label='{0:d} hojas'.format(hh))
    aa[0].set_title('Entrenamiento')
    aa[1].set_title('Test')
for jj in [0,1]:
  aa[jj].set_xlabel('Num. arboles')
  aa[jj].set_xscale('log')
  aa[jj].legend()
  aa[jj].set_ylabel('Acierto (%)')
  aa[jj].grid()      
  
ff.tight_layout()

ff.show()
print("La mejor configuración en test es {0:d} árboles de {1:d} hojas, acierto del {2:.2f}%".format(best_num_arboles, best_num_hojas, best_test))
acierto_spam_rf = best_test

## Random Forest y  Housing


In [ ]:
housing = fetch_california_housing()
data = housing.data[:nn_california]
targets = housing.target[:nn_california]
data_df = pd.DataFrame(np.vstack((data.T,targets)).T, columns=housing['feature_names']+ housing['target_names'])
x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.3, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(max_leaf_nodes=6,
                           n_estimators=100)
RF.fit(x_train, y_train)
riesgo_entrenamiento = RF.score(x_train, y_train)
riesgo_test = RF.score(x_test, y_test)
print("R^2 en el conjunto de entrenamiento: {0:.2f}".format(riesgo_entrenamiento))
print("R^2 en el conjunto de test: {0:.2f}".format(riego_test))

In [ ]:
ff,aa = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12,8))
best_test = -10000
best_num_arboles = -1
best_num_hojas = -1
for kk, hh in enumerate(v_hojas): 
    dt_entr = np.empty(len(v_arboles))
    dt_test = np.empty(len(v_arboles))
    for ih,B in enumerate(v_arboles):
        RF = RandomForestRegressor(max_leaf_nodes=hh,
                       n_estimators=B)
        RF.fit(x_train, y_train)
        dt_entr[ih] = RF.score(x_train, y_train)
        dt_test[ih] = RF.score(x_test, y_test)
    if np.max(dt_test) > best_test:
      best_test = np.max(dt_test)
      best_num_arboles = B
      best_num_hojas = hh       
    aa[0].plot(v_arboles, dt_entr, label='{0:d} hojas'.format(hh))
    aa[1].plot(v_arboles, dt_test, label='{0:d} hojas'.format(hh))
    aa[0].set_title('Entrenamiento')
    aa[1].set_title('Test')
for jj in [0,1]:
    aa[jj].set_xlabel('Num. arboles')
    aa[jj].set_xscale('log')
    aa[jj].legend()
    aa[jj].grid()
    aa[jj].set_ylabel("$R^2$")
    
        
ff.tight_layout()

print("La mejor configuración en test es {0:d} árboles de {1:d} hojas, R2 del {2:.2f}".format(best_num_arboles, best_num_hojas, best_test))
acierto_housing_rf = best_test

Podemos observar cómo es más crítico el valor del número de hojas que el del número de árboles a la hora de determinar el éxito del modelo

# Resumen de los mejores métodos

In [ ]:
metodos_spam = ['baseline','kNN', 'kNN ponderado','arboles decision','random forest ']
resultados_spam = [baseline_spam*100.,acierto_spam_knn, acierto_spam_knn_w, acierto_spam_dt,acierto_spam_rf]
fx, ax = plt.subplots(1,1, figsize=(8,4))
_ = ax.bar(range(len(metodos_spam)),
       width=0.5,
       height=resultados_spam,
       tick_label=metodos_spam)
ax.set_ylabel('Acierto (%)')
_=ax.set_title('Resultados Spam Filtering')

In [ ]:
metodos_spam = ['baseline','kNN', 'kNN ponderado','arboles decision','random forest ']
resultados_spam = [0, acierto_housing_knn, acierto_housing_knn_w, acierto_housing_dt,acierto_housing_rf]
fx, ax = plt.subplots(1,1,figsize=(8,4))
_ = ax.bar(range(len(metodos_spam)),
       width=0.5,
       height=resultados_spam,
       tick_label=metodos_spam)
ax.set_ylabel('$R^2$')
_=ax.set_title('Resultados Housing')

## Puntos para discutir la idoneidad de las soluciones
- Precisión 
- Explicabilidad de los resultados
- Capacidad de generalización
- Mantenimiento del modelo
- Tiempo de entrenamiento y de test


## Visualización de las fronteras de decisión en el *toy problem* 

### Para el $k$NN

In [ ]:
v_hh = [1,2,3,4,5,6]
fx,ax = plt.subplots(len(v_hh),2,sharex=True, sharey=True, figsize=(13,30))
cc=0
plot_step=0.05
x_min, x_max = x0[:, 0].min() - 0.1, x0[:, 0].max() + 0.1
y_min, y_max = x0[:, 1].min() - 0.1, x0[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

y1 = y.copy()
y1[y1==0] = -1
for cc,kk in enumerate(v_hh):
  clf = KNeighborsClassifier(n_neighbors=kk)
  clf.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc][0])
  ax[cc][0].set_xlabel('x1')
  ax[cc][0].set_ylabel('x2')
  ax[cc][0].set_title("{0:d} vecinos".format(kk))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc][0].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc][0].set_ylim([y_min, y_max])

  clf = KNeighborsClassifier(n_neighbors=kk, weights='distance')
  clf.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc][1])
  ax[cc][1].set_xlabel('x1')
  ax[cc][1].set_ylabel('x2')
  ax[cc][1].set_title("{0:d} vecinos, ponderados".format(kk))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc][1].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc][1].set_ylim([y_min, y_max])

### Árboles de decisión con varios números de hoja


In [ ]:
v_hh = [2,3,4,6,8,10]
fx,ax = plt.subplots(len(v_hh),1,sharex=True, sharey=True, figsize=(6,30))
cc=0
plot_step=0.05
x_min, x_max = x0[:, 0].min() - 0.1, x0[:, 0].max() + 0.1
y_min, y_max = x0[:, 1].min() - 0.1, x0[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

y1 = y.copy()
y1[y1==0] = -1
for cc,kk in enumerate(v_hh):
  DT = DecisionTreeClassifier(max_leaf_nodes=kk,random_state=42)
  DT.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc])
  ax[cc].set_xlabel('x1')
  ax[cc].set_ylabel('x2')
  ax[cc].set_title('{0:d} hojas'.format(kk))
  Z = DT.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc].set_ylim([y_min, y_max])


### Random Forests

In [ ]:
v_hh = [2,5,10,20]
fx,ax = plt.subplots(len(v_hh),2,sharex=True, sharey=True, figsize=(13,30))
cc=0
plot_step=0.05
x_min, x_max = x0[:, 0].min() - 0.1, x0[:, 0].max() + 0.1
y_min, y_max = x0[:, 1].min() - 0.1, x0[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

y1 = y.copy()
y1[y1==0] = -1
for cc,kk in enumerate(v_hh):
  hojas=2
  clf = RandomForestClassifier(max_leaf_nodes=hojas,
                       n_estimators=kk)
  clf.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc][0])
  ax[cc][0].set_xlabel('x1')
  ax[cc][0].set_ylabel('x2')
  ax[cc][0].set_title("{0:d} arboles, {1:d} hojas".format(kk, hojas))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc][0].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc][0].set_ylim([y_min, y_max])

  hojas=6
  clf = RandomForestClassifier(max_leaf_nodes=hojas,
                       n_estimators=kk)
  clf.fit(x, -y1)
  pinta_ejemplo(x,y,xt,ax[cc][1])
  ax[cc][1].set_xlabel('x1')
  ax[cc][1].set_ylabel('x2')
  ax[cc][1].set_title("{0:d} arboles, {1:d} hojas".format(kk, hojas))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)
  cs = ax[cc][1].contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.2)
  _ = ax[cc][1].set_ylim([y_min, y_max])